Initializing Google Earth Engine and importing geemap

In [5]:
import ee
import geemap
import json
import os
from geemap import geojson_to_ee
from ipyleaflet import GeoJSON

In [6]:
Map = geemap.Map()
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [ ]:
# May not be necessary

#ee.Authenticate()
#ee.Initialize(project = 'ee-jdawsey')

In [9]:
#landcover = ee.Image('USGS/NLCD/NLCD2016').select('landcover')
#Map.addLayer(landcover, {}, 'Landcover Map 2016')

https://github.com/gee-community/geemap/blob/master/examples/notebooks/07_geojson.ipynb

In [ ]:
file_path = os.path.abspath('C:/Users/Justin-Laptop/Documents/Research/GEE_LPC/mesq_jsons')

"""if not os.path.exists(file_path):
    url = 'https://github.com/gee-community/geemap/raw/master/examples/data/us_states.json'
    r = requests.get(url)
    with open(file_path, 'w') as f:
        f.write(r.content.decode("utf-8"))"""

files_list = [] # need to add list of files still.

def file_address_func(file_given):
    addresses = []
    for file in file_path:
        address = "C:/Users/Justin-Laptop/Documents/Research/GEE_LPC/mesq_jsons/"
        full_address = address + file
        length_list = len(addresses)
        addresses = addresses.append(full_address)
    return addresses

listy = file_address_func(files_list)

with open(file_path) as f:
    json_data = json.load(f)

Importing each geojson for use in GEE

In [ ]:
ee_object = geemap.geojson_to_ee(geojson_file_path)

exportTask = ee.batch.Export.table.toAsset(
    collection = ee_object,
    description = 'Circular polygon for segmentation and classification',
    assetId = 'users/username/folderName'
)
exportTask.start()

Loading in each polygon for data to get clipped to.

In [ ]:
shp = '' # list

year = ee.ImageCollection('USDA/NAIP/DOQQ')
    .filterdate('2016-01-01', '2016-12-31')
    .filterbounds(shp)

clip = year.clip(shp)

visParam = [bands = ...]

map.centerObject(shp)
map.addLayer(clip, visParams = visParam)

Adding indices for additional possible accuracy

SAVI (Soil Adjusted Vegetation Index):
SAVI = (1 + L) * (Bnir - Bred) / (Bnir + Bred + L)
--- L = correction factor (0 = high veg, 1 = very little veg)

In [ ]:
def rgb_savi(image_given):
    image_bands = image_given.select("R", "N")
    image = image_bands

    savi = image.expression(
        expression = '(1 + 0.6) * (N - R) / (N + R + 0.6)',
        opt_map = [
            'R' = image.select('R'),
            'N' = image.select('N')
        ]
    ).rename('savi')
    return(savi)

savi = rgb_savi(clip)
#map.addLayer(savi)

clip = clip.addBands(savi)

#checking was added
#band_names = standardized.bandNames()
#print(band_names.get(4).getInfo())

ENDVI (Enhanced Normalized Difference Vegetation Index):
ENDVI = ((NIR + Green) - (2 * Blue)) / ((NIR + Green) + (2 * Blue))

In [ ]:
def rgb_endvi(image_given):
    image_bands = image_given.select("N", "G", "B")
    image = image_bands

    endvi = image.expression(
        expression = '((N + G) - (2 * B)) / ((N + G) + (2 + B))',
        opt_map = ['N' = image.select('N'),
                   'G' = image.select('G'),
                   'B' = image.select('B')
                   ]
    ).rename('endvi')
    return(endvi)

endvi = rgb_endvi(clip)

clip = clip.addBands(endvi)

Standardization

In [ ]:
image_st = clip
st_bandNames = image_st.bandNames()
meanDict = image_st.reduceRegion(
  reducer = ee.Reducer.mean(),
  geometry = shp,
  scale = 1,
  maxPixels = 200000000,
  bestEffort = TRUE,
  tileScale = 16
)
means = ee.Image.constant(meanDict.values(st_bandNames))
centered = image_st.subtract(means)

stdDevDict = image_st.reduceRegion(
  reducer = ee.Reducer.stdDev(),
  geometry = shp,
  scale = 1,
  maxPixels = 200000000,
  bestEffort = TRUE,
  tileScale = 16
)

stddevs = ee.Image.constant(stdDevDict.values(st_bandNames))
standardized = centered.divide(stddevs)

visParam_st = [bands = c('N', 'R', 'G'),
                    min = -1,
                    max = 2,
                    gamma = 1.2
]

Smoothing and sharpening

In [ ]:
### gaussian smoothing
gaussianKernel = ee.Kernel.gaussian(
  radius = 3,
  units = 'pixels'
)

gaussian_smooth = standardized.convolve(gaussianKernel)


#visParamGauss <- list(bands <- c('R', 'G', 'B'),
#  min= 0,
#  max= 255
#  )

### checking that image was smoothed
#Map$addLayer(gauss, visParams = visParamGauss)



### DoG sharpening
fat = ee.Kernel.gaussian(
  radius = 3,
  sigma = 3,
  magnitude = -1.0,
  units = 'pixels'
)

skinny = ee.Kernel.gaussian(
  radius = 3,
  sigma = 0.5,
  units = 'pixels'
)

dog = fat.add(skinny)
#changed name from sharpened
gauss = gaussian_smooth.add(standardized.convolve(dog)) 

In [ ]:
segmented = ee.Algorithms.Image.Segmentation.SNIC(gauss)
Map.addLayer(segmented)

Training the classification algorithm and showing the display parameters

In [ ]:
### creating training samples for the unsupervised classification
training = segmented.sample(
  region = shp,
  scale = 1, #change depending on year
  numPixels = 10000
)

### the actual classification function
clusterer = ee.Clusterer.wekaXMeans(maxClusters = 12, # change clusters depending on imagery
                                    maxIterations = 5, 
                                    useKD = TRUE).ee.Clusterer.train(training)

result = segmented.cluster(clusterer)

